In [1]:
# Import Libraries
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from ta.others import daily_return,cumulative_return
from plotly.offline import init_notebook_mode
import cufflinks as cf
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import requests
from bs4 import BeautifulSoup
import warnings
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
%matplotlib inline
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
# Check if folder input exists

# Define the folder name
folder_name = '../input'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../input' folder already exists.


In [3]:
# Check if folder tickers exists

# Define the folder name
folder_name = '../tickers'

# Check if the folder exists
if not os.path.exists(folder_name):
    # If it doesn't exist, create the folder
    os.makedirs(folder_name)
    print(f"'{folder_name}' folder created.")
else:
    print(f"'{folder_name}' folder already exists.")

'../tickers' folder already exists.


In [4]:
# Webscrapping Wikipedia table to get the DAX40 components
page = requests.get("https://en.wikipedia.org/wiki/MDAX")
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find('table',id='constituents',class_="wikitable sortable zebra")

In [5]:
# Convert html table to dataframe
wiki = pd.read_html(str(table))
wiki = pd.concat(wiki)
wiki.head(40)

,Logo,Name,Industry,Index weighting,Number of Shares[6],Free-float- in Mio. €,Location,Symbol
0,NaN,Aixtron SE,Semiconductor industry,298.00,0113.402.370,"4.325,62",Herzogenrath,AIXA
1,NaN,Aroundtown S.A.,Real estate,1.49,1536397797,2061.72,Luxemburg,AT1
2,NaN,Aurubis AG,Metals,159.00,0044.956.723,"2.307,09",Hamburg,NDA
3,NaN,Bechtle AG,IT services,257.00,0126.000.000,"3.741,68",Neckarsulm,BC8
4,NaN,Befesa S.A.,Waste management,1.11,NaN,1530.47,Ratingen,BFSA
5,NaN,Bilfinger SE,NaN,NaN,NaN,NaN,Mannheim,GBF
6,NaN,Carl Zeiss Meditec AG,Medical technology,245.00,0089.440.570,"3.555,14",Jena,AFX
7,NaN,CTS Eventim AG & Co. KGaA,Leisure-events,256.00,0096.000.000,"3.713,64",München,EVD
8,NaN,Delivery Hero,Delivery service,423.00,0269.536.421,"6.136,97",Berlin,DHER
9,NaN,Deutsche Lufthansa AG,Airlines,569.00,1.195.485.644,"8.267,54",Köln,LHA


In [6]:
wiki['Symbol'] = wiki['Symbol'].apply(lambda x: x + '.DE')
wiki

,Logo,Name,Industry,Index weighting,Number of Shares[6],Free-float- in Mio. €,Location,Symbol
0,NaN,Aixtron SE,Semiconductor industry,298.00,0113.402.370,"4.325,62",Herzogenrath,AIXA.DE
1,NaN,Aroundtown S.A.,Real estate,1.49,1536397797,2061.72,Luxemburg,AT1.DE
2,NaN,Aurubis AG,Metals,159.00,0044.956.723,"2.307,09",Hamburg,NDA.DE
3,NaN,Bechtle AG,IT services,257.00,0126.000.000,"3.741,68",Neckarsulm,BC8.DE
4,NaN,Befesa S.A.,Waste management,1.11,NaN,1530.47,Ratingen,BFSA.DE
5,NaN,Bilfinger SE,NaN,NaN,NaN,NaN,Mannheim,GBF.DE
6,NaN,Carl Zeiss Meditec AG,Medical technology,245.00,0089.440.570,"3.555,14",Jena,AFX.DE
7,NaN,CTS Eventim AG & Co. KGaA,Leisure-events,256.00,0096.000.000,"3.713,64",München,EVD.DE
8,NaN,Delivery Hero,Delivery service,423.00,0269.536.421,"6.136,97",Berlin,DHER.DE
9,NaN,Deutsche Lufthansa AG,Airlines,569.00,1.195.485.644,"8.267,54",Köln,LHA.DE


In [7]:
wiki.rename(columns={'Industry':'Sector','Symbol':'Ticker'}, inplace=True)

In [8]:
# Add index row values
new_row = {'Name': 'DAXMidcap', 'Sector': 'Index', 'Ticker': '^MDAXI'}

# Convert the new row to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Append the new row to the DataFrame
wiki = pd.concat([wiki[['Name','Sector','Ticker']], new_row_df], ignore_index=True)
wiki.head(41)

,Name,Sector,Ticker
0,Aixtron SE,Semiconductor industry,AIXA.DE
1,Aroundtown S.A.,Real estate,AT1.DE
2,Aurubis AG,Metals,NDA.DE
3,Bechtle AG,IT services,BC8.DE
4,Befesa S.A.,Waste management,BFSA.DE
5,Bilfinger SE,NaN,GBF.DE
6,Carl Zeiss Meditec AG,Medical technology,AFX.DE
7,CTS Eventim AG & Co. KGaA,Leisure-events,EVD.DE
8,Delivery Hero,Delivery service,DHER.DE
9,Deutsche Lufthansa AG,Airlines,LHA.DE


In [9]:
# Get the data for the stock index
index_list = wiki['Ticker'].tolist()

In [10]:
# Save all the historical data
for stock in tqdm(index_list):
    data = yf.download(stock, progress=False,multi_level_index=False,actions=True,auto_adjust=False)
    data.to_csv(f"../input/{stock}.csv",index=True)

  0%|          | 0/51 [00:00<?, ?it/s]

In [11]:
# Get the name of the notebook
notebook_name = os.path.basename(globals()['__vsc_ipynb_file__'])
notebook_name = notebook_name.split('-')[0]

In [12]:
# Save all the tickers data
wiki[['Name','Sector','Ticker']].to_csv(f"../tickers/{notebook_name}.csv",index=True)

In [13]:
# Configuration of different parameters of the notebook
ticker = 'AFX.DE'
year = '2025'

In [14]:
# Check DataFrame
stock_ticker = pd.read_csv(f"../input/{ticker}.csv",index_col="Date",parse_dates=True)
stock_ticker.head(10)

,Adj Close,Close,Dividends,High,Low,Open,Stock Splits,Volume
Date,,,,,,,,
2000-03-22,20.176100,25.564892,0.0,26.806616,24.841770,25.418806,0.0,195685
2000-03-23,27.093620,34.329998,0.0,35.060425,27.025743,28.778765,0.0,304837
2000-03-24,28.246538,35.790848,0.0,43.095104,34.695210,38.347340,0.0,244324
2000-03-27,25.940697,32.869148,0.0,37.616913,32.138721,37.251701,0.0,85064
2000-03-28,24.038380,30.458742,0.0,35.783546,29.954750,32.869148,0.0,54543
2000-03-29,22.654877,28.705723,0.0,31.043083,28.705723,30.677872,0.0,48514
2000-03-30,23.634857,29.947445,0.0,29.947445,26.989222,28.398943,0.0,68917
2000-03-31,24.880009,31.525164,0.0,32.321327,27.836515,28.778765,0.0,72930
2000-04-03,24.413078,30.933519,0.0,33.563053,30.020489,32.576977,0.0,40021


In [15]:
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':3,'colspan':1}],[None],[None],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Adj Close'],mode="lines",name=f"{ticker}"),row=1, col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker.index,y=stock_ticker['Volume'],mode="lines",name='Volume'),row=4, col=1)
fig.update_yaxes(title_text="Volume", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker}")

fig.show()

In [16]:
# Group per year and calculate cummulative return
stock_ticker['Year'] = stock_ticker.index.year
annual_data = stock_ticker.groupby('Year').agg(Adj_Close=('Adj Close','last'))
annual_data['Return'] = annual_data["Adj_Close"].pct_change()*100
annual_data

,Adj_Close,Return
Year,,
2000,12.393890,NaN
2001,5.188139,-58.139541
2002,5.274609,1.666670
2003,7.153866,35.628372
2004,7.897501,10.394872
2005,10.376279,31.386864
2006,12.564589,21.089542
2007,9.578944,-23.762373
2008,7.094030,-25.941417


In [17]:
# Plotly
fig = make_subplots()

# Add colors
colors = ['green' if x >= 0 else 'red' for x in annual_data['Return']]

# Graph (1,1)
fig.add_trace(go.Bar(y=annual_data['Return'],x=annual_data['Return'].index,text=round(annual_data['Return'],2),textposition='outside',marker_color=colors,hoverinfo='skip'),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Returns", row=1, col=1)

fig.update_layout(xaxis=dict(tickvals=annual_data.index,tickangle=-45),height=800, width=1300,title=f"Returns per year {ticker}")

fig.show()

In [18]:
# Add Technical Analysis Indicators

# Modified Moving Average 20
stock_ticker['MMA20'] = stock_ticker['Adj Close'].loc[year].rolling(20).mean() #Adj Close 20 MA

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=stock_ticker["Adj Close"].loc[year], window=20, window_dev=2)

# Bollinger Bands
stock_ticker['BB_Upper'] = indicator_bb.bollinger_hband()
stock_ticker['BB_Lower'] = indicator_bb.bollinger_lband()

# Initialize RSI Indicator
indicator_rsi = RSIIndicator(close=stock_ticker["Adj Close"].loc[year], window=14)

# RSI
stock_ticker['RSI'] = indicator_rsi.rsi()

# Daily Return
stock_ticker['Daily_Return'] = daily_return(stock_ticker["Adj Close"].loc[year])

In [19]:
# Plot the adjusted close price
fig = make_subplots(rows=4, cols=1,shared_xaxes=True,vertical_spacing=0.01,specs=[[{'rowspan':2,'colspan':1}],[None],[{'rowspan':1,'colspan':1}],[{'rowspan':1,'colspan':1}]])

# Graph (1,1)
fig.add_trace(go.Scatter(x=stock_ticker['Adj Close'].loc[year].index,y=stock_ticker['Adj Close'].loc[year],mode="lines",name=f'{ticker}'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Lower'].loc[year].index,y=stock_ticker['BB_Lower'].loc[year],mode="lines",name='BB_Lower'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['BB_Upper'].loc[year].index,y=stock_ticker['BB_Upper'].loc[year],mode="lines",name='BB_Upper'),row=1,col=1)
fig.add_trace(go.Scatter(x=stock_ticker['MMA20'].loc[year].index,y=stock_ticker['MMA20'].loc[year],mode="lines",name='MMA20'),row=1,col=1)
# Update xaxis properties
fig.update_yaxes(title_text="Price", row=1, col=1)

# Graph (3,1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year],mode="lines",name='Volume'),row=3, col=1)
fig.add_trace(go.Scatter(x=stock_ticker['Volume'].loc[year].index,y=stock_ticker['Volume'].loc[year].rolling(20).mean(),mode="lines",name='MMA20'),row=3,col=1)
fig.update_yaxes(title_text="Volume", row=3, col=1)

# Graph (4,1)
fig.add_trace(go.Scatter(x=stock_ticker['RSI'].loc[year].index,y=stock_ticker['RSI'].loc[year],mode="lines",name='RSI'),row=4, col=1)
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="green",row=4,col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="red",row=4,col=1)
fig.update_yaxes(title_text="RSI", row=4, col=1)

fig.update_layout(height=800, width=1300,showlegend=False,title=f"{ticker} {year}")

fig.show()

In [20]:
# Plotly
fig = make_subplots()

# Graph (1,1)
# Loop all stock files and get cummulative return for year
for stock in tqdm(index_list):
    f = os.path.join("../input", stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    fig.add_trace(go.Scatter(x=df['Cummulative_Return'].loc[year].index,y=df['Cummulative_Return'].loc[year],mode="lines",name=stock.split('.')[0]),row=1,col=1)

# Update xaxis properties
fig.update_yaxes(title_text="Return", row=1, col=1)

fig.update_layout(height=800, width=1300,showlegend=True,title=f"Cummulative Returns {ticker} for {year}")

fig.show()

  0%|          | 0/51 [00:00<?, ?it/s]

In [21]:
# Create also table of cummulative returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Cummulative_Return'] = cumulative_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Cummulative_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Cummulative_Return'])
cum = wiki[['Ticker','Name','Sector']].merge(cum,on='Ticker')
cum.sort_values(by=['Cummulative_Return'],ignore_index=True, ascending=False).head(60)


  0%|          | 0/51 [00:00<?, ?it/s]

,Ticker,Name,Sector,Date,Cummulative_Return
0,TKA.DE,Thyssenkrupp AG,Conglomerate,2025-03-20,145.956974
1,HAG.DE,Hensoldt AG,NaN,2025-03-20,101.064461
2,GBF.DE,Bilfinger SE,NaN,2025-03-20,51.086957
3,AFX.DE,Carl Zeiss Meditec AG,Medical technology,2025-03-20,40.893463
4,NDX1.DE,Nordex SE,NaN,2025-03-20,39.489357
5,KGX.DE,Kion Group AG,Handling equipment,2025-03-20,39.318399
6,JUN3.DE,Jungheinrich AG,"Intralogistics, Mechanical engineering",2025-03-20,34.848484
7,HOT.DE,Hochtief AG,Construction,2025-03-20,31.571001
8,BFSA.DE,Befesa S.A.,Waste management,2025-03-20,28.952381
9,EVK.DE,Evonik Industries AG,Chemistry,2025-03-20,28.507325


In [22]:
# Create also table of daily returns
list = []

for stock in tqdm(index_list):
    f = os.path.join("../input",stock)
    df = pd.read_csv(f+".csv",index_col="Date",parse_dates=True)
    df['Daily_Return'] = daily_return(df["Adj Close"].loc[year])
    list.append([df.loc[year].tail(1).index.item(),stock.split('.csv')[0],df["Daily_Return"].loc[year].iloc[-1]])


cum = pd.DataFrame(list, columns=['Date','Ticker','Daily_Return'])
cum = wiki[['Ticker','Name']].merge(cum,on='Ticker')
cum.sort_values(by=['Daily_Return'],ignore_index=True, ascending=False).head(40)


  0%|          | 0/51 [00:00<?, ?it/s]

,Ticker,Name,Date,Daily_Return
0,FRA.DE,Fraport AG,2025-03-20,2.473501
1,SAX.DE,Ströer SE & Co. KGaA,2025-03-20,0.956520
2,AT1.DE,Aroundtown S.A.,2025-03-20,0.901633
3,BOSS.DE,Hugo Boss AG,2025-03-20,0.872088
4,TLX.DE,Talanx AG,2025-03-20,0.624023
5,EVD.DE,CTS Eventim AG & Co. KGaA,2025-03-20,0.597013
6,TEG.DE,TAG Immobilien AG,2025-03-20,0.479612
7,LEG.DE,LEG Immobilien AG,2025-03-20,0.456626
8,SDF.DE,K+S AG,2025-03-20,0.442637
9,RAA.DE,Rational AG,2025-03-20,0.177830
